For this project, you're going to select a dataset of your choosing and create a classification model. You'll start by identifying a problem you can solve with classification, and then identify a dataset. You'll then use everything you've learned about Data Science and Machine Learning thus far to source a dataset, preprocess and explore it, and then build and interpret a classification model that answers your chosen question.

steps
1. import libraries/packages
2. import data
3. inspect data
4. clean data 
4a. do I need to convert lat/long to location? 
        if no, drop lat, long and is_geocoding_exact
5. standardize? 
6. scaling/transformation
7. normalize? 
8. test/train split and random state
9. vanilla tree
9A. feature importance 
10. model performance and accuracy
grid search
examine best param
11. random forests
bagged trees 

pipelines

need dummies
    pd.get_dummies
create armed or no variable? 
dummies for whole dataset or just bool expressions? 
group by states 

https://github.com/cphalpert/census-regions
https://github.com/washingtonpost/data-police-shootings

### 1. Importing Necessary Libraries 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor  
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix as CM
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import chisquare
import sys
import warnings
warnings.filterwarnings("ignore")

if i decide to come back and map coordinates 

import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook

### 2. Importing Datasets, Merging, and Inspecting Data

In [ ]:
df = pd.read_csv('fatal-police-shootings-data.csv')

In [ ]:
df.head()

Importing State Classification Dataset to Minimize Classification Variables 

In [ ]:
states = pd.read_csv('us census bureau regions and divisions.csv')
states.head()

In [ ]:
states = states.drop(['State'], axis=1)

In [ ]:
states = states.rename(columns={"State Code": "state"})

In [ ]:
data = pd.merge(states, df, on='state')
data.head()

dropping name and id column 

In [ ]:
data = data.drop(['name', 'id', 'state', 'region'], axis=1)

In [ ]:
data.info()

converting bool to int 
dropping geocoding rows with inaccurate lat/long info 

### 3. Cleaning and Converting Data

In [ ]:
data = data[data.is_geocoding_exact != 'False']

In [ ]:
data = data.applymap(lambda x: 1 if x == True else x)
data = data.applymap(lambda x: 0 if x == False else x)

In [ ]:
data.isna().sum()

dropping nan values and filling others 

In [ ]:
data = data.dropna(subset=['age', 'longitude', 'latitude', 'flee'])

In [ ]:
values = {'race': 'U', 'armed': 'undetermined'}
data = data.fillna(value=values)

In [ ]:
data.isna().sum()

In [ ]:
display(data.describe())

In [ ]:
data['threat_level'].unique()
data['flee'].unique()

In [ ]:
data['manner_of_death'].unique()

create new armed column? armed with lethal weapon - true/false? or unarmed v. armed? 

In [ ]:
data['armed'].value_counts()

#### 3A. Examining Armed Values Prior to Conversion

In [ ]:
pie_me = data['armed'].value_counts().reset_index()
pie_me.columns = ['armed', 'count']

fig = px.pie(
    pie_me, 
    values='count', 
    names='armed',  
    title='Weapon distribution', 
    width=600, 
    height=600
)

fig.show()

In [ ]:
#converting armed values with a count of less than 59 into 'other'
data[['armed']] = data[['armed']].where(data.apply(lambda x: x.groupby(x).transform('count')>59), 'other')

In [ ]:
data['armed'].value_counts()

### 4. Initial Analysis and EDA 1 of 

In [ ]:
chisquare(data.signs_of_mental_illness)

In [ ]:
pd.crosstab(data.armed, data.signs_of_mental_illness, normalize='index')

In [ ]:
pd.crosstab(data.race, data.body_camera, normalize='index')

In [ ]:
sns.catplot(x="signs_of_mental_illness", kind="count", palette="ch:.25", data=data)

In [ ]:
sns.catplot(x="signs_of_mental_illness", y="flee", hue="gender", kind="point", data=data)

In [ ]:
sns.catplot(y="signs_of_mental_illness", hue="threat_level", kind="count",
            palette="pastel", edgecolor=".6",
            data=data)

creating df without state, region, city, long, lat, is geocoding exact, and date

to include or not include lat, long, geocoding? 

In [ ]:
no_locdf = data.drop(['city'], axis=1)
no_locdf.head()

In [ ]:
dummydf = pd.get_dummies(no_locdf)
dummydf.head()

In [ ]:
classifier_ls = ['Logistic_Regression', 'Random_Forest', 'Gradient_Boosting', 'K_NN']
classifier_dc = {
               'Logistic_Regression': LogisticRegression(class_weight='auto'),
               'Random_Forest': RandomForestClassifier(n_estimators=50, class_weight='balanced'),
               'Gradient_Boosting': GradientBoostingClassifier(max_depth=5),
               'K_NN': KNeighborsClassifier(n_neighbors=5, weights='distance'),
               }
n_trials = 25
test_size_rt = 0.2

In [ ]:
y = dummydf['signs_of_mental_illness']
X = dummydf.drop('signs_of_mental_illness', axis=1)

taking too long - how to take a sample of the data? 

In [ ]:
score_ls = []
for ic, (cl_name, Classifier) in enumerate(classifier_dc.items()):
    for trial in range(n_trials):
        train_signals, test_signals, train_labels, test_labels = train_test_split(X, y, test_size=test_size_rt, random_state=42)
        Classifier.fit(train_signals, train_labels)
        pred_labels = Classifier.predict(test_signals)
        precision = precision_score(test_labels, pred_labels, average='binary')
        score_ls.append([cl_name, precision, 'Precision'])
        recall = recall_score(test_labels, pred_labels, average='binary')
        score_ls.append([cl_name, recall,'Recall'])
        F1 = f1_score(test_labels, pred_labels, average='binary')
        score_ls.append([cl_name, F1,'f1 Score'])


In [ ]:
plt.figure(figsize=(16, 6))
metrics_df = pd.DataFrame(score_ls, columns=['Classifier','Score','Score_type']) #building a dataframe with various metrics
ax = sns.violinplot(x="Classifier", y="Score", hue="Score_type", data=metrics_df, linewidth=2) #producing the boxplot


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

initial decision tree analysis 

In [ ]:
regressor = DecisionTreeRegressor(random_state=42)  
regressor.fit(X_train, y_train)  

create scatterplot to examine features 

MSE lower with location data omitted 

In [ ]:
y_pred = regressor.predict(X_test) 

print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
dt_clf = DecisionTreeClassifier()

In [ ]:
dt_cv_score = cross_val_score(dt_clf, X_train, y_train, cv=3)
mean_dt_cv_score = np.mean(dt_cv_score)

print(f"Mean Cross Validation Score: {mean_dt_cv_score :.2%}")

depth can lead to overfitting

In [ ]:
dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6]
}

In [ ]:
# Instantiate GridSearchCV
dt_grid_search = GridSearchCV(dt_clf, dt_param_grid, cv=3, return_train_score=True)

# Fit to the data
dt_grid_search.fit(X_train, y_train)

In [ ]:
# Mean training score
dt_gs_training_score = np.mean(dt_grid_search.cv_results_['mean_train_score'])

# Mean test score
dt_gs_testing_score = dt_grid_search.score(X_test, y_test)

print(f"Mean Training Score: {dt_gs_training_score :.2%}")
print(f"Mean Test Score: {dt_gs_testing_score :.2%}")
print("Best Parameter Combination Found During Grid Search:")
dt_grid_search.best_params_

In [ ]:
rf_clf = RandomForestClassifier()
mean_rf_cv_score = np.mean(cross_val_score(rf_clf, X_train, y_train, cv=3))

print(f"Mean Cross Validation Score for Random Forest Classifier: {mean_rf_cv_score :.2%}")

In [ ]:
rf_param_grid = {
    'n_estimators': [10, 30, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 6, 10],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [3, 6]
}

In [ ]:
rf_grid_search = GridSearchCV(rf_clf, rf_param_grid, cv=3)
rf_grid_search.fit(X_train, y_train)

print(f"Training Accuracy: {rf_grid_search.best_score_ :.2%}")
print("")
print(f"Optimal Parameters: {rf_grid_search.best_params_}")

In [ ]:
dt_score = dt_grid_search.score(X_test, y_test)
rf_score = rf_grid_search.score(X_test, y_test)

print('Decision tree grid search: ', dt_score)
print('Random forest grid search: ', rf_score)

In [ ]:
#y = df['signs_of_mental_illness']
#X = df.drop('signs_of_mental_illness', axis=1)


In [ ]:
#ddf = pd.get_dummies(df, prefix=['signs_of_mental_illness', 'body_camera', ''])

In [ ]:
sns.catplot(y="flee", hue="signs_of_mental_illness", kind="count",
            palette="pastel", edgecolor=".6",
            data=df)

In [ ]:
sns.lineplot(x='threat_level', y='age', data=df, hue='signs_of_mental_illness')

In [ ]:
f, ax = plt.subplots(figsize=(7, 5))
sns.despine(f)
sns.distplot(df['age'], bins=20)

In [ ]:
plt.figure(figsize=(15,12))
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
sns.heatmap(corr, mask=mask,
        xticklabels=corr.columns,
        yticklabels=corr.columns, cmap='RdBu', annot = True)